In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import scipy.optimize as opt
import sys, os, glob
import matplotlib.pyplot as plt
import plotly.graph_objs as go

In [3]:
# load data into data frames

dataDir = '.\data\\'

dataFrames = {}
values = ['recovered', 'deaths', 'confirmed']
for file in glob.glob(os.path.join(dataDir, '*.csv')):
    for value in values:
        if value in file:
            dataFrames[value] = pd.read_csv(file, index_col=[1,0], header=[0])

In [4]:
# set current file
current_frame_name = 'confirmed'
df = dataFrames[current_frame_name]

In [4]:
# basic information
df.head()

,,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,5/31/21,6/1/21,6/2/21,6/3/21,6/4/21,6/5/21,6/6/21,6/7/21,6/8/21,6/9/21
Country/Region,Province/State,,,,,,,,,,,,,,,,,,,,,
Afghanistan,NaN,33.93911,67.709953,0,0,0,0,0,0,0,0,...,71838,72977,74026,75119,76628,77963,79224,80841,82326,84050
Albania,NaN,41.15330,20.168300,0,0,0,0,0,0,0,0,...,132315,132337,132351,132360,132372,132374,132379,132384,132397,132415
Algeria,NaN,28.03390,1.659600,0,0,0,0,0,0,0,0,...,128913,129218,129640,129976,130361,130681,130958,131283,131647,132034
Andorra,NaN,42.50630,1.521800,0,0,0,0,0,0,0,0,...,13727,13729,13744,13752,13758,13758,13758,13777,13781,13791
Angola,NaN,-11.20270,17.873900,0,0,0,0,0,0,0,0,...,34551,34752,34960,35140,35307,35594,35772,35854,36004,36115


In [20]:
# simple plotting
import plotly.graph_objects as go
countries_to_plot = ['Germany', 'France', 'Italy']

df_cntrs = df.loc[pd.IndexSlice[countries_to_plot,:],:]['Data'].T
print(df_cntrs.index.get_level_values(0))
#print(df_cntrs['France'].columns.values)
#df_cntrs = df_cntrs.set_index('Country/Region').T
#print(df_cntrs[df_cntrs['index'])
first = True
fig = go.Figure()
if True:
    for country in countries_to_plot:
        if first:
            #print(df_cntrs[country].columns)
            if len(df_cntrs[country].columns.values) > 1:
                fig = px.line(x=[a for a in df_cntrs.index.get_level_values(0)], y=df_cntrs[country].to_numpy()[:,0])
                fig.data[-1].name=df_cntrs[country].columns.values[0]
                fig.data[-1].showlegend = True
            
                for i in range(1, len(df_cnts[country].columns.values)):
                    fig.add_scatter(x=df_cntrs.index, y=df_cntrs[country].to_numpy()[:,i], mode='lines')
                    fig.data[-1].name=country + ", " + df_cntrs[country].columns.values[i]
            else:
                #print(df_cntrs[country][2:].to_numpy()[:,0])
                #print([a[1] for a in df_cntrs.index.get_level_values(0)])
                fig = px.line(x=[a for a in df_cntrs.index.get_level_values(0)], y=df_cntrs[country].to_numpy()[:,0])
                fig.data[-1].name=country + ", " + df_cntrs[country].columns.values[0]
                fig.data[-1].showlegend = True
                pass
            first = False
        else:
            if len(df_cntrs[country].columns.values) > 1:
                for i in range(len(df_cntrs[country].columns.values)):
                    fig.add_scatter(x=[a for a in df_cntrs.index.get_level_values(0)], y=df_cntrs[country].to_numpy()[:,i], mode='lines')
                    fig.data[-1].name=country + ", " + df_cntrs[country].columns.values[i]
            else:
                #print(df_cntrs[country][2:].to_numpy()[:,0])
                fig.add_scatter(x=[a for a in df_cntrs.index.get_level_values(0)], y=df_cntrs[country].to_numpy()[:,0], mode='lines', name=country + ", " + df_cntrs[country].columns.values[0])
    #df_cntrs.plot()
    fig.show()

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       '1/28/20', '1/29/20', '1/30/20', '1/31/20',
       ...
       '5/28/21', '5/29/21', '5/30/21', '5/31/21', '6/1/21', '6/2/21',
       '6/3/21', '6/4/21', '6/5/21', '6/6/21'],
      dtype='object', length=502)


In [5]:
# cleanup and feature engineering

def cleanup(df):
    new_df = df.copy(deep=True) #  copy dataframe so original is not changed
    cols = pd.MultiIndex.from_tuples([('Coords', 'Lat'), ('Coords', 'Lon')]+list(zip(['Data']*len(df.columns.values[2:]), df.columns.values[2:]))) # add multilevel for numerical data
    new_df.columns = cols # update columns multilevel
    new_df.index = pd.MultiIndex.from_frame(df.index.to_frame().fillna('Main')) # replace NA in Regions with Main
    return new_df

def total(df):
    df['Total', 'tot'] = df['Data'][df['Data'].columns.values[-1]]
    df.loc[('World','main'),:] = df.sum(axis=0)
    df.loc[('World', 'main')]['Coords'] = (np.nan, np.nan)
    return df


test_df = cleanup(df)

test_df['Total', 'tot'] = test_df['Data'][test_df['Data'].columns.values[-1]]
test_df.loc[('World','main'),:] = test_df.sum(axis=0)
test_df.loc[('World', 'main')]['Coords'] = (np.nan, np.nan)
print(test_df['Data'].loc[('World', 'main')][-1])
#print(len(list(test_df['Data'].columns.values[::20])), len(list(test_df['Data'].columns.values[::20])+['Total']))

174420543.0


In [6]:
df2 = total(cleanup(dataFrames['confirmed']))
dfrecov = total(cleanup(dataFrames['recovered']))
dataFrames['active'] = df2.copy()
dataFrames['active']['Data']  -= dfrecov['Data']
dataFrames['active']['Total']  -= dfrecov['Total']
df_diff = dataFrames['active']['Data'].diff(periods=1, axis=1)
dataFrames['active_diff'] = df_diff.drop(df_diff.columns[0], axis=1)

In [9]:
dFrames = {}
for v in values:
    df = dataFrames[v]
    df = total(cleanup(df))
    dFrames[v] = df

def curve(x, a, b):
    return a*x + b


ratioDF = (dFrames['deaths']['Data']/dFrames['confirmed']['Data'])


xvals0 = dFrames['confirmed']['Total', 'tot'].iloc[:-1].copy()
yvals0 = dFrames['deaths']['Total', 'tot'].iloc[:-1].copy()

print(yvals0['China'])

dropmask0 = yvals0[yvals0 <= 0].index.values
xvals0 = xvals0.drop(dropmask0)
yvals0 = yvals0.drop(dropmask0)

dropmask00 = xvals0[xvals0 <= 0].index.values
xvals0 = xvals0.drop(dropmask00)
yvals0 = yvals0.drop(dropmask00)


names0 = xvals0.index.unique().values

dropmask1 = xvals0[xvals0 > 1e7].index.values
yvals1 = yvals0.drop(dropmask1)
xvals1 = xvals0.drop(dropmask1)
names1 = xvals1.index.unique().values

dropmask2 = xvals1[xvals1 > 2e4].index.values
yvals2 = yvals1.drop(dropmask2)
xvals2 = xvals1.drop(dropmask2)
names2 = xvals2.index.unique().values

def filterZeros(x, y):
    x[x <= 0] = 1
    y[y <= 0] = 1
    return x, y

#xvals0, yvals0 = [np.log(i) for i in filterZeros(xvals0, yvals0)]
#xvals1, yvals1 = [np.log(i) for i in filterZeros(xvals1, yvals1)]
#xvals2, yvals2 = [np.log(i) for i in filterZeros(xvals2, yvals2)]
#xvals0, yvals0 = filterZeros(xvals0, yvals0)
#xvals1, yvals1 = filterZeros(xvals1, yvals1)
#xvals2, yvals2 = filterZeros(xvals2, yvals2)

popt0, pcov0 = opt.curve_fit(curve, np.log(xvals0), np.log(yvals0), bounds=([-np.inf, -np.inf], [np.inf, 0]))
popt1, pcov1 = opt.curve_fit(curve, np.log(xvals1), np.log(yvals1), bounds=([-np.inf, -np.inf], [np.inf, 0]))
popt2, pcov2 = opt.curve_fit(curve, np.log(xvals2), np.log(yvals2), bounds=([-np.inf, -np.inf], [np.inf, 0]))
print(popt0, popt1, popt2)

fig = px.scatter(x=xvals0, y=yvals0, hover_name=names0, log_x = True, log_y = True)
fig.add_trace(go.Scatter(x=[np.amin(xvals0), np.amax(xvals0)], y=[np.exp(curve(np.log(i),*popt0)) for i in [np.amin(xvals0), np.amax(xvals0)]], mode='lines', name='All Countries'))
fig.add_trace(go.Scatter(x=[np.amin(xvals0), np.amax(xvals0)], y=[np.exp(curve(np.log(i),*popt1)) for i in [np.amin(xvals0), np.amax(xvals0)]], mode='lines', name='#confirmed < 1e7'))
fig.add_trace(go.Scatter(x=[np.amin(xvals0), np.amax(xvals0)], y=[np.exp(curve(np.log(i),*popt2)) for i in [np.amin(xvals0), np.amax(xvals0)]], mode='lines', name='#confirmed < 2e4'))
fig.update_layout(
    xaxis_title = '# Confirmed Cases',
    yaxis_title = '# Deaths'
)

fig.show()

Province/State
Anhui                6.0
Beijing              9.0
Chongqing            6.0
Fujian               1.0
Gansu                2.0
Guangdong            8.0
Guangxi              2.0
Guizhou              2.0
Hainan               6.0
Hebei                7.0
Heilongjiang        13.0
Henan               22.0
Hong Kong          210.0
Hubei             4512.0
Hunan                4.0
Inner Mongolia       1.0
Jiangsu              0.0
Jiangxi              1.0
Jilin                3.0
Liaoning             2.0
Macau                0.0
Ningxia              0.0
Qinghai              0.0
Shaanxi              3.0
Shandong             7.0
Shanghai             7.0
Shanxi               0.0
Sichuan              3.0
Tianjin              3.0
Tibet                0.0
Unknown              0.0
Xinjiang             3.0
Yunnan               2.0
Zhejiang             1.0
Name: (Total, tot), dtype: float64
[ 1.07304473 -5.07877321] [ 1.07490678 -5.09582628] [ 0.98033144 -4.40657251]


In [17]:
test_df['Data'].loc['World'][test_df['Data'].columns.values[-1]]

Province/State
main    174420543.0
Name: 6/9/21, dtype: float64

In [79]:
datsize =test_df['Total', 'tot'].iloc[:-1]
print(str(datsize.index.values[0]))

('Afghanistan', 'Main')


In [72]:
px.scatter_geo(lat=test_df['Coords', 'Lat'].iloc[:-1].to_numpy(), lon=test_df['Coords', 'Lon'].iloc[:-1].to_numpy(), size=[i for i in test_df['Total', 'tot'].iloc[:-1].to_numpy()], color=(test_df['Total', 'tot'].iloc[:-1].to_numpy()), hover_name=test_df.iloc[:-1].index.values, opacity=0.8, projection='natural earth', title='Worldwide')